In [1]:
import pandas as pd
import os
import re

In [1108]:
def converttojson(df, dfname):# Convert DataFrame to JSON
    json_data = df.to_json(orient='records')
    file_path = f"Clean Data/{dfname}.json"

    # Optionally, save to a file if needed
    with open(file_path, 'w') as f:
        f.write(json_data)
    
    print(f"DataFrame '{dfname}' saved as JSON file: {file_path}")

def firstrowtocolname(df):
    column_names = df.iloc[0]
    # Remove the first row from the DataFrame
    df = df[1:]
    # Set the extracted row as column names
    df.columns = column_names
    # Reset the index after removing the first row
    df.reset_index(drop=True, inplace=True)
    return df

def fingertipsclean(df):
    df= df[['AreaName','Time period', 'Value']]
    df['AreaName'] = df['AreaName'].replace('London region', 'London')
    df_filtered = df[-6:] #take the last six rows (2 years of data)
    return df_filtered

def censusclean(df):
    new_columns=[df.columns[0]]
    for i, col in enumerate(df.columns):
        if pd.isna(col):
            new_col_name = f"{df.columns[i-1]} (%)"
            new_columns.append(new_col_name)
        elif i != 0:
            new_col_name = f"{df.columns[i]} (number)"
            new_columns.append(new_col_name)
    df.columns = new_columns
    df = df.iloc[1:]
    return df
def crimerate(rawdf, offencegroup='', subgroup=''):
    rawdf = rawdf.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df = rawdf[rawdf['Area name'].isin(wards+['Lambeth']) & 
            (rawdf['Measure']=='Offences')]
    if offencegroup != '':
        df = df[(df['Offence Group'] == offencegroup)]
    if subgroup != '':
        df= df[df['Offence Subgroup'] == subgroup]
    
    df = df.iloc[:,[0,3,10]]
    column_mapping = {df.columns[0]: 'Time period', df.columns[1]: 'AreaName'}
    df = df.rename(columns=column_mapping)
    df['Count'] = pd.to_numeric(df['Count'], errors='coerce')
    df = df.groupby(['Time period','AreaName'], as_index=False).sum()

    londondf = rawdf[(rawdf['Area Type']=='Borough') & (~rawdf['Area name'].str.contains('Aviation')) & 
            (rawdf['Measure']=='Offences')]
    if offencegroup != '':
        londondf = londondf[(londondf['Offence Group'] == offencegroup)]
    if subgroup!= '':
        londondf=londondf[londondf['Offence Subgroup'] == subgroup]
    londondf = londondf.iloc[:,[0,3,10]]
    column_mapping = {londondf.columns[0]: 'Time period', londondf.columns[1]: 'AreaName'}
    londondf = londondf.rename(columns=column_mapping)
    londondf['Count'] = pd.to_numeric(londondf['Count'], errors='coerce')
    londondf = londondf.groupby(['Time period'], as_index=False).sum()
    londondf['AreaName'] = 'London'
    combineddf = pd.concat([df, londondf], ignore_index=True)
    combineddf = pd.merge(pop, combineddf, on='AreaName', how='inner')
    combineddf['Time period'] = combineddf['Time period'].str.replace(r'-01$', '', regex=True)    
    combineddf['Population'] = pd.to_numeric(combineddf['Population'], errors='coerce')
    combineddf['Offences per 1,000 people'] = (combineddf['Count']/combineddf['Population'] * 1000).round(1)
    combineddf=combineddf.drop(columns=['Female','Male'])
    return combineddf

def create_surv_q_df(df, start_text="test col x vs y and z", end_text_select=1):
    surv_sat = df.iloc[:, :37]
    indices_to_drop = list(range(3, 12))
    surv_sat = surv_sat.drop(surv_sat.columns[indices_to_drop], axis=1)
    question = str(surv_sat.iloc[0,0])

    names = surv_sat.iloc[3, 2:].values
    # Create boolean masks with no NaN values
    start_mask = surv_sat.iloc[:,0].str.contains(start_text, regex=False, na=False)
    end_mask = surv_sat.iloc[:,0].str.contains("Don’t know|Prefer not to say|NET", regex=True, na=False)

    # Find start and end indices for filtering
    if not start_mask.any() or not end_mask.any():
        raise ValueError("Start or end text not found in 'Response' column.")

    start_index = surv_sat[start_mask].index[0] + 1
    end_indices = surv_sat[end_mask].index
    if len(end_indices) < end_text_select:
        raise ValueError("Insufficient matches for end text to use the specified end_text_select value.")
    end_index = end_indices[end_text_select - 1] - 1

    surv_sat_f = surv_sat.loc[start_index:end_index]

    columns=['Response','Percentage of respondents']
    columns.extend(names)

    # Rename the rest of the columns to 'Percentage of respondents'
    column_mapping = {old_name: new_name for old_name, new_name in zip(surv_sat_f.columns, columns)}
    surv_sat_f = surv_sat_f.rename(columns=column_mapping)
    surv_sat_f = surv_sat_f.dropna(subset=[surv_sat_f.columns[0]])

    # Replace numbers in square brackets in the 'Response' column
    surv_sat_f['Response'] = surv_sat_f['Response'].str.replace(r"\[\d+\] ", "", regex=True)

    # Convert 'Percentage of respondents' to numeric and handle any non-numeric gracefully
    surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

    # Multiply 'Percentage of respondents' by 100 to get percentage and round to 1 decimal place
    surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:] * 100
    surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:].round(1)

    # Make column 'Question' with values from the question variable
    surv_sat_f['Question'] = question

    # Various string replacements and clean-ups on 'Question'
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace("Q:", "")
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"^Q[A-Za-z0-9]+", "", regex=True)
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"^_[A-Za-z0-9]+", "", regex=True)
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"\.", " ")
    surv_sat_f['Question'] = surv_sat_f['Question'].str.strip()
    return surv_sat_f

def create_surv_q_df2022(df, start_text="test col x vs y and z", end_text_select=1):
    surv_sat = df.iloc[:, :2]

    question = str(surv_sat.iloc[0,0])

    # Create boolean masks with no NaN values
    start_mask = surv_sat.iloc[:,0].str.contains(start_text, regex=False, na=False)
    end_mask = surv_sat.iloc[:,0].str.contains("Don’t know|Prefer not to say|NET", regex=True, na=False)

    # Find start and end indices for filtering
    if not start_mask.any() or not end_mask.any():
        raise ValueError("Start or end text not found in 'Response' column.")

    start_index = surv_sat[start_mask].index[0] + 1
    end_indices = surv_sat[end_mask].index
    if len(end_indices) < end_text_select:
        raise ValueError("Insufficient matches for end text to use the specified end_text_select value.")
    end_index = end_indices[end_text_select - 1] - 1

    surv_sat_f = surv_sat.loc[start_index:end_index]

    columns=['Response','Percentage of respondents']

    # Rename the rest of the columns to 'Percentage of respondents'
    column_mapping = {old_name: new_name for old_name, new_name in zip(surv_sat_f.columns, columns)}
    surv_sat_f = surv_sat_f.rename(columns=column_mapping)
    surv_sat_f = surv_sat_f.dropna(subset=[surv_sat_f.columns[0]])

    # Replace numbers in square brackets in the 'Response' column
    surv_sat_f['Response'] = surv_sat_f['Response'].str.replace(r"\[\d+\] ", "", regex=True)

    
    # Convert 'Percentage of respondents' to numeric and handle any non-numeric gracefully
    surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

    # Multiply 'Percentage of respondents' by 100 to get percentage and round to 1 decimal place
    surv_sat_f.iloc[:, 1] = surv_sat_f.iloc[:, 1] * 100
    surv_sat_f.iloc[:, 1] = surv_sat_f.iloc[:, 1].round(1)

    # Make column 'Question' with values from the question variable
    surv_sat_f['Question'] = question

    # Various string replacements and clean-ups on 'Question'
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace("Q:", "")
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"^Q[A-Za-z0-9]+", "", regex=True)
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"^_[A-Za-z0-9]+", "", regex=True)
    surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"\.", " ")
    surv_sat_f['Question'] = surv_sat_f['Question'].str.strip()
    return surv_sat_f

In [1041]:
# Read the Excel file containing file names of datasets
data_sources = pd.read_excel('Data sources ambitions.xlsx')

In [486]:
wards = ["Brixton Acre Lane","Brixton North","Brixton Rush Common","Brixton Windrush",
         "Clapham Common & Abbeville","Clapham East","Clapham Park","Clapham Town","Gipsy Hill",
         "Herne Hill & Loughborough Junction","Kennington","Knight's Hill","Myatt's Fields","Oval",
         "St Martin's","Stockwell East","Stockwell West & Larkhall","Streatham Common & Vale",
         "Streatham Hill East","Streatham Hill West & Thornton","Streatham St Leonard's",
         "Streatham Wells","Vauxhall","Waterloo & South Bank","West Dulwich"]

## Import current data

In [1181]:
dataframes = {}
# Iterate over the rows to get the file paths
for index, row in data_sources.iterrows():
    if row['exported?'] == 'y': # comment out when re-importing everything
        continue
    if pd.isna(row['File Name']):
        continue
    if row['Ambition'] == 'Golden Thread':
        ambition_folder = '..\Data\Golden Thread\\'
    else:
        ambition_folder = '..\Data\Ambition '+ str(row['Ambition']) + '\\' + str(row['Section'])
    
    file_name = str(row['File Name'])
    variable_name = row['Internal link suffix']
    sheet_name = str(row['Sheet']) if 'Sheet' in row and pd.notna(row['Sheet']) else 0
    skip = int(row['Skip rows']) if 'Skip rows' in row and pd.notna(row['Skip rows']) else None
    numrows = int(row['nrows']) if 'nrows' in row and pd.notna(row['nrows']) else None
    file_path = os.path.join(ambition_folder, file_name)
    print(file_path)

    # Load the data (example for CSV)
    if file_path.endswith('.csv'):
        # Try reading the CSV file with different encodings
        encodings_to_try = ['utf-8', 'latin1', 'ISO-8859-1', 'cp1252', 'utf-16']
        for encoding in encodings_to_try:
            try:
                dataframes[variable_name] = pd.read_csv(file_path, header=None, skiprows=skip, encoding = encoding)
                print(f"CSV file successfully read with encoding: {encoding}")
                break  # Stop trying encodings if successful
            except UnicodeDecodeError:
                print(f"Failed to read CSV file with encoding: {encoding}") 
        
        # Perform operations on the data
    elif file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        dataframes[variable_name] = pd.read_excel(file_path, sheet_name = sheet_name, header=None, 
                                                  skiprows=skip, nrows=numrows)
    elif file_path.endswith('.ods'):
        dataframes[variable_name] = pd.read_excel(file_path, engine = 'odf', sheet_name = sheet_name, header=None, skiprows=skip)
        # Perform operations on the data
    if row['File Name'] == '2023 ARS crosstabs.xlsx':
        dataframes[variable_name] = create_surv_q_df(dataframes[variable_name])
    elif 'OtherCrime' in row['File Name']:
        dataframes[variable_name] = firstrowtocolname(dataframes[variable_name])
        column_mapping = {'Crime Type': 'Offence Group', 'Crime Subtype':'Offence Subgroup', 'Area Name' : 'Area name'}
        dataframes[variable_name] = dataframes[variable_name].rename(columns=column_mapping)
        
    else:
        dataframes[variable_name] = firstrowtocolname(dataframes[variable_name])

# uncomment when running everything again
    if row['Source organisation'] == 'Fingertips':
        dataframes[variable_name]= fingertipsclean(dataframes[variable_name])
        converttojson(dataframes[variable_name], variable_name)
    elif row['Source organisation'] == 'ONS':
        dataframes[variable_name] = dataframes[variable_name][:-3] #remove last three rows
        dataframes[variable_name] = dataframes[variable_name].dropna(how='all')
    elif row['Source organisation'] == 'ONS Census':
        # Remove anything before ":" in row names
        dataframes[variable_name].iloc[:,0] = dataframes[variable_name].iloc[:,0].str.split(':').str[-1]
        dataframes[variable_name] = dataframes[variable_name][:-2] #remove last two rows
        dataframes[variable_name] = dataframes[variable_name].dropna(how='all')
        dataframes[variable_name].columns = dataframes[variable_name].columns.str.replace(r'^.*:', '')
        dataframes[variable_name].columns = dataframes[variable_name].columns.str.replace(r'\s*\(Lambeth\)', '', regex=True)
        dataframes[variable_name] = dataframes[variable_name].rename(columns={'Area': 'AreaName'})
        if 'AreaName' in dataframes[variable_name].columns:
            dataframes[variable_name]['AreaName'] = dataframes[variable_name]['AreaName'].str.replace(r'\s*\(Lambeth\)', '', regex=True)
        if dataframes[variable_name].iloc[0].str.contains('number').any():
            dataframes[variable_name]= censusclean(dataframes[variable_name])
        converttojson(dataframes[variable_name], variable_name)

..\Data\Ambition 1\Health and Wellbeing\2023 ARS crosstabs.xlsx


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:109: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:122: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"\.", " ")


..\Data\Ambition 1\Environment\GreenHouseGasEmissions.csv
CSV file successfully read with encoding: utf-8
..\Data\Ambition 1\Environment\Recycling.ods
..\Data\Ambition 1\Environment\EPCRatings.ods
..\Data\Ambition 1\Environment\EPCRatings.ods
..\Data\Ambition 1\Environment\EPCRatings.ods
..\Data\Ambition 1\Environment\SolarPanelInstallations.xlsx
..\Data\Ambition 1\Environment\osprivateoutdoorspacereferencetables.xlsx
..\Data\Ambition 1\Environment\osprivateoutdoorspacereferencetables.xlsx
..\Data\Ambition 1\Environment\osprivateoutdoorspacereferencetables.xlsx
..\Data\Ambition 1\Environment\TreeCanopyCover.csv
CSV file successfully read with encoding: utf-8
..\Data\Ambition 2\Crime, Safety & Justice\MPS_MonthlyCrimeDashboard_TNOCrimeData23-24.csv


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\3201268869.py:27: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes[variable_name] = pd.read_csv(file_path, header=None, skiprows=skip, encoding = encoding)
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\3201268869.py:27: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes[variable_name] = pd.read_csv(file_path, header=None, skiprows=skip, encoding = encoding)


CSV file successfully read with encoding: utf-8
..\Data\Ambition 2\Crime, Safety & Justice\MPS_MonthlyCrimeDashboard_OtherCrimeData23-24.csv
CSV file successfully read with encoding: utf-8
..\Data\Ambition 2\Crime, Safety & Justice\MPS_MonthlyCrimeDashboard_OtherCrimeData23-24.csv


...

..\Data\Golden Thread\2023 ARS crosstabs.xlsx


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:122: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"\.", " ")
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:109: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:122: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"\.", " ")


..\Data\Golden Thread\2023 ARS crosstabs.xlsx
..\Data\Golden Thread\2023 ARS crosstabs.xlsx


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:109: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:122: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"\.", " ")


..\Data\Golden Thread\CBP7096-trends-by-country-and-region.xlsx
..\Data\Golden Thread\Poverty rates by London borough (2021_22).csv
CSV file successfully read with encoding: utf-8
..\Data\Golden Thread\children-in-low-income-families-local-area-statistics-2014-to-2023.ods
..\Data\Golden Thread\Child-Poverty-AHC-estimates-2015-2022_final.xlsx
..\Data\Golden Thread\Child-Poverty-AHC-estimates-2015-2022_final.xlsx
..\Data\Golden Thread\Pension Credit.xlsx
..\Data\Golden Thread\electoralstatstables2023v2.xlsx
..\Data\Golden Thread\2022 England Electoral Turnout data.xlsx
..\Data\Golden Thread\2022 England Electoral Turnout data.xlsx


In [1044]:
pop = dataframes['population-by-sex']
pop['Population'] = pop['Female'] +pop['Male']

In [1028]:
dataframes.keys()

dict_keys(['childhood-vaccinations', 'healthy-weight---reception', 'healthy-weight---year-6', 'good-level-of-development---age-5', 'wellbeing---resident-survey-worthwhile', 'wellbeing---resident-survey-anxiety', 'personal-wellbeing', 'sexual-health', 'physical-exercise', 'healthy-weight', 'smoking', 'high-blood-pressure', 'drug-and-alcohol-misuse', 'diabetes', 'depression', 'coronary-heart-disease', 'male-life-expectancy-at-birth', 'female-life-expectancy-at-birth', 'male-healthy-life-expectancy', 'female-healthy-life-expectancy', 'deaths-under-75-from-preventable-causes', 'preventable-mortality---cancer', 'preventable-mortality---cardiovascular-disease', 'preventable-mortality---liver-disease', 'preventable-mortality---respiratory-disease', 'greenhouse-gas-emissions', 'recycling-rates', 'energy-performance-certificate-ratings-Lambeth', 'energy-performance-certificate-ratings-London', 'energy-performance-certificate-ratings-England', 'solar-panel-installations', 'access-to-public-green

## Import last year data (if available)

In [ ]:
lydataframes = {}
# Iterate over the rows to get the file paths
for index, row in data_sources.iterrows():
    if row['exported?'] == 'y': # comment out when re-importing everything
        continue
    if pd.isna(row['Previous year File Name']) or row['File Name'] == row['Previous year File Name']:
        continue
    if row['Ambition'] == 'Golden Thread':
        ambition_folder = '..\Data\Golden Thread\\'
    else:
        ambition_folder = '..\Data\Ambition '+ str(row['Ambition']) + '\\' + str(row['Section'])
    
    file_name = str(row['Previous year File Name'])
    variable_name = 'lastyear' + row['Internal link suffix']
    if 'Previous year sheet' in row and pd.notna(row['Previous year sheet']):
        sheet_name = str(row['Previous year sheet'])
    else:
        sheet_name = str(row['Sheet']) if 'Sheet' in row and pd.notna(row['Sheet']) else 0
    skip = int(row['Skip rows']) if 'Skip rows' in row and pd.notna(row['Skip rows']) else None
    numrows = int(row['nrows']) if 'nrows' in row and pd.notna(row['nrows']) else None
    file_path = os.path.join(ambition_folder, file_name)
    print(file_path)
    
    # Load the data (example for CSV)
    if file_path.endswith('.csv'):
        # Try reading the CSV file with different encodings
        encodings_to_try = ['utf-8', 'latin1', 'ISO-8859-1', 'cp1252', 'utf-16']
        for encoding in encodings_to_try:
            try:
                lydataframes[variable_name] = pd.read_csv(file_path, header=None, skiprows=skip, encoding = encoding)
                print(f"CSV file successfully read with encoding: {encoding}")
                break  # Stop trying encodings if successful
            except UnicodeDecodeError:
                print(f"Failed to read CSV file with encoding: {encoding}") 
        
        # Perform operations on the data
    elif file_path.endswith('.xlsx') or file_path.endswith('.xls'):
        lydataframes[variable_name] = pd.read_excel(file_path, sheet_name = sheet_name, header=None, 
                                                  skiprows=skip, nrows=numrows)
    elif file_path.endswith('.ods'):
        lydataframes[variable_name] = pd.read_excel(file_path, engine = 'odf', sheet_name = sheet_name, header=None, skiprows=skip)
        # Perform operations on the data
    if row['Previous year File Name'] == 'res_survey_2022_filt.xlsx':
        lydataframes[variable_name] = create_surv_q_df2022(lydataframes[variable_name])
    elif 'OtherCrime' in row['Previous year File Name']:
        lydataframes[variable_name] = firstrowtocolname(lydataframes[variable_name])
        column_mapping = {'Crime Type': 'Offence Group', 'Crime Subtype':'Offence Subgroup', 'Area Name' : 'Area name'}
        lydataframes[variable_name] = lydataframes[variable_name].rename(columns=column_mapping)
    else:
        lydataframes[variable_name] = firstrowtocolname(lydataframes[variable_name])

# uncomment when running everything again
    if row['Source organisation'] == 'Fingertips':
        dataframes[variable_name]= fingertipsclean(dataframes[variable_name])
        converttojson(dataframes[variable_name], variable_name)
    elif row['Source organisation'] == 'ONS':
        lydataframes[variable_name] = lydataframes[variable_name][:-3] #remove last three rows
        lydataframes[variable_name] = lydataframes[variable_name].dropna(how='all')

..\Data\Ambition 1\Health and Wellbeing\res_survey_2022_filt.xlsx


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:159: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:172: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"\.", " ")


..\Data\Ambition 2\Crime, Safety & Justice\MPS_MonthlyCrimeDashboard_TNOCrimeData22-23.csv


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\3850455025.py:30: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  lydataframes[variable_name] = pd.read_csv(file_path, header=None, skiprows=skip, encoding = encoding)


CSV file successfully read with encoding: utf-8
..\Data\Ambition 2\Crime, Safety & Justice\MPS_MonthlyCrimeDashboard_OtherCrimeData22-23.csv


...

..\Data\Ambition 3\Jobs, earnings, and businesses\Qualifications NOMIS 2022.xlsx
..\Data\Ambition 3\Jobs, earnings, and businesses\Work Geography LWF Table 7 LWF.1b   lwfmgx 2022.xls
..\Data\Ambition 3\Jobs, earnings, and businesses\Unemployment 16+ Sex and Ethnicity 2022.xlsx
..\Data\Ambition 3\Community\res_survey_2022_filt.xlsx
..\Data\Ambition 3\Community\res_survey_2022_filt.xlsx
..\Data\Ambition 3\Community\res_survey_2022_filt.xlsx
..\Data\Ambition 3\Community\res_survey_2022_filt.xlsx


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:159: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:172: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"\.", " ")
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:159: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)

..\Data\Ambition 3\Community\res_survey_2022_filt.xlsx
..\Data\Ambition 3\Community\res_survey_2022_filt.xlsx
..\Data\Ambition 3\Community\res_survey_2022_filt.xlsx
..\Data\Ambition 3\Community\res_survey_2022_filt.xlsx
..\Data\Golden Thread\res_survey_2022_filt.xlsx


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:159: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  surv_sat_f.iloc[:, 1:] = surv_sat_f.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:172: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f['Question'] = surv_sat_f['Question'].str.replace(r"\.", " ")
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:159: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)

..\Data\Golden Thread\res_survey_2022_filt.xlsx
..\Data\Golden Thread\res_survey_2022_filt.xlsx
..\Data\Golden Thread\Electoral data - Local-Elections - May 2018.ods


In [1032]:
lydataframes.keys()

dict_keys(['lastyearwellbeing---resident-survey-worthwhile', 'lastyearwellbeing---resident-survey-anxiety', 'lastyeardrug-and-alcohol-misuse', 'lastyearoverall-crime-rate', 'lastyearviolent-crime-rate', 'lastyearknife-crime-rate', 'lastyearknife-crime-rate-with-victims-younger-than-25', 'lastyeardomestic-abuse-crime-rate', 'lastyearhate-crime-rate', 'lastyeardrug-related-crime-rate', 'lastyeardomestic-burglary-crime-rate', 'lastyearresident-survey-perception-of-safety-day', 'lastyearresident-survey-perception-of-safety-evening', 'lastyearresident-survey-police-confidence', 'lastyeartypes-of-industry', 'lastyearqualifications', 'lastyearweekly-earnings-by-sex', 'lastyearearning-a-living-wage', 'lastyearunemployment-by-sex-and-ethnicity', 'lastyearresident-survey-results---council-satisfaction-of-life', 'lastyearresident-survey-results---council-runs-things', 'lastyearresident-survey-results---council-value', 'lastyearresident-survey-results---council-informed', 'lastyearresident-survey-

# Wrangling

In [1122]:
dataframes.keys()

dict_keys(['childhood-vaccinations', 'healthy-weight---reception', 'healthy-weight---year-6', 'good-level-of-development---age-5', 'wellbeing---resident-survey-worthwhile', 'wellbeing---resident-survey-anxiety', 'personal-wellbeing', 'sexual-health', 'physical-exercise', 'healthy-weight', 'smoking', 'high-blood-pressure', 'drug-and-alcohol-misuse', 'diabetes', 'depression', 'coronary-heart-disease', 'male-life-expectancy-at-birth', 'female-life-expectancy-at-birth', 'male-healthy-life-expectancy', 'female-healthy-life-expectancy', 'deaths-under-75-from-preventable-causes', 'preventable-mortality---cancer', 'preventable-mortality---cardiovascular-disease', 'preventable-mortality---liver-disease', 'preventable-mortality---respiratory-disease', 'greenhouse-gas-emissions', 'recycling-rates', 'energy-performance-certificate-ratings-Lambeth', 'energy-performance-certificate-ratings-London', 'energy-performance-certificate-ratings-England', 'solar-panel-installations', 'access-to-public-green

In [1179]:
dataframes['children-in-low-income-households-BHC-Lambeth']

,"Table 3: Number and Percentage of Children (aged under 16) living in Relative low income families, Local Authority, FYE 2015 to 2023, United Kingdom",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Back to Contents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,This worksheet contains one table. Some cells ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Some shorthand is used in this table, [p] = pr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,This table shows children aged 0 to 15 years o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FYE = Financial Years Ending.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,Fermanagh and Omagh,N09000006,5035,5049,4968,4545,4721,4920,5543,4819,4935,0.203,0.204,0.201,0.183,0.189,0.197,0.223,0.195,0.199
367,Lisburn and Castlereagh,N09000007,3742,3729,3732,3458,3737,4075,4274,3708,3893,0.135,0.133,0.132,0.121,0.128,0.137,0.144,0.123,0.128
368,Mid and East Antrim,N09000008,4731,4948,4841,4466,4709,5032,5244,4488,4695,0.181,0.19,0.185,0.17,0.178,0.191,0.2,0.173,0.182
369,Mid Ulster,N09000009,6496,6536,6184,5550,5826,6143,7107,6263,6491,0.197,0.197,0.185,0.164,0.171,0.178,0.206,0.181,0.187


In [499]:
dataframes['types-of-industry'] #, 'qualifications', 'weekly-earnings-by-sex', 'earning-a-living-wage', 'unemployment-by-sex-and-ethnicity', 'unemployment-benefit-claimants', 'distance-to-work']

,Industry,lacu:Lambeth,country:England,gor:London
0,"01 : Crop and animal production, hunting and r...",10,90260,640
1,02 : Forestry and logging,5,3420,230
2,03 : Fishing and aquaculture,0,1725,45
3,05 : Mining of coal and lignite,0,10,0
4,06 : Extraction of crude petroleum and natural...,0,100,40
...,...,...,...,...
87,99 : Activities of extraterritorial organisati...,0,5,5
88,Column Total,14960,2737105,584415
89,NaN,NaN,NaN,NaN
90,All figures are rounded to avoid disclosure. V...,NaN,NaN,NaN


# Completed

In [1177]:
#Poverty rate AHC ------------------------------------------
# 5year avg data from 2016/17-2021/22 excluding 2020/21
lambethdf = dataframes['poverty-rate-lambeth']
lambethdf=lambethdf[lambethdf['London borough']=='Lambeth'].loc[:,['London borough', 'Poverty rate (AHC)']]
lambethdf = lambethdf.rename(columns={'London borough':'Area name', 'Poverty rate (AHC)': 'Percent'})
lambethdf['Time period']= '2016/17-2021-22'
lambethdf['Percent'] = int(lambethdf['Percent'].str.replace("%", ""))

df = dataframes['poverty-rate-londonengland']

df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df = df[df['Region'].isin(['London', 'England']) &
       (df['Group']=='Total')&
       (df['Poverty measure']=='Relative')]

df = df[(df['Quantity']=='%') &
        (df['Housing costs']=='AHC')]
df = df.loc[:,['Region', '2019/20 - 2021/22', '2020/21 - 2022/23']]
df
melted_df = pd.melt(df, id_vars=['Region'], var_name='Time period', value_name='Percent')
# 
melted_df = melted_df.rename(columns={'Region':'Area name'})

combineddf = pd.concat([lambethdf, melted_df], ignore_index=True)
converttojson(combineddf, 'poverty-rate-AHC')
combineddf

DataFrame 'poverty-rate-AHC' saved as JSON file: Clean Data/poverty-rate-AHC.json


,Area name,Percent,Time period
0,Lambeth,28,2016/17-2021-22
1,England,22,2019/20 - 2021/22
2,London,25,2019/20 - 2021/22
3,England,22,2020/21 - 2022/23
4,London,24,2020/21 - 2022/23


In [1121]:
# Perception of Safety Resident survey ----------------------------------------------------------
q015_1 = dataframes['resident-survey-perception-of-safety-day']
q015_2= dataframes['resident-survey-perception-of-safety-evening']
q1y22 = dataframes['resident-survey-police-confidence']
q2bnew22 = dataframes['resident-survey-harassment-hate-crime']

# Join all DataFrames
surv_sat_f_safety = pd.concat([q015_1, q015_2, q1y22, q2bnew22], ignore_index=True)

# Add a 'Year' column with value 2023
surv_sat_f_safety['Time period'] = 2023
surv_sat_f_safety.drop(surv_sat_f_safety.columns[2], axis=1, inplace=True)

# Remove anything between square brackets in Responses
surv_sat_f_safety['Response'] = surv_sat_f_safety['Response'].str.replace(r'\[.*\]', '')

# Make 'Response' an ordered categorical column with levels in the order of appearance
surv_sat_f_safety['Response'] = pd.Categorical(surv_sat_f_safety['Response'], categories=surv_sat_f_safety['Response'].unique(), ordered=True)
converttojson(surv_sat_f_safety,'survey_wellbeing_safety')
surv_sat_f_safety

# Last Year Perception of Safety Resident survey ----------------------------------------------------------
q015_1 = lydataframes['lastyearresident-survey-perception-of-safety-day']
q015_2= lydataframes['lastyearresident-survey-perception-of-safety-evening']
q1y22 = lydataframes['lastyearresident-survey-police-confidence']

# Join all DataFrames
lastyearsurv_sat_f_safety = pd.concat([q015_1, q015_2, q1y22], ignore_index=True)

# Add a 'Year' column with value 2022
lastyearsurv_sat_f_safety['Time period'] = 2022

# Remove anything between square brackets in Responses
lastyearsurv_sat_f_safety['Response'] = lastyearsurv_sat_f_safety['Response'].str.replace(r'\[.*\]', '')

# Make 'Response' an ordered categorical column with levels in the order of appearance
lastyearsurv_sat_f_safety['Response'] = pd.Categorical(lastyearsurv_sat_f_safety['Response'], categories=lastyearsurv_sat_f_safety['Response'].unique(), ordered=True)
converttojson(lastyearsurv_sat_f_safety,'lastyearsurvey_wellbeing_safety')
lastyearsurv_sat_f_safety

DataFrame 'survey_wellbeing_safety' saved as JSON file: Clean Data/survey_wellbeing_safety.json
DataFrame 'lastyearsurvey_wellbeing_safety' saved as JSON file: Clean Data/lastyearsurvey_wellbeing_safety.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1209231986.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f_safety['Response'] = surv_sat_f_safety['Response'].str.replace(r'\[.*\]', '')
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1209231986.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
  lastyearsurv_sat_f_safety['Response'] = lastyearsurv_sat_f_safety['Response'].str.replace(r'\[.*\]', '')


,Response,Percentage of respondents,Question,Time period
0,Very safe,47.9,"To what extent would you say you are, or would...",2022
1,Fairly safe,44.2,"To what extent would you say you are, or would...",2022
2,Not particularly safe,5.9,"To what extent would you say you are, or would...",2022
3,Not safe at all,1.3,"To what extent would you say you are, or would...",2022
4,Very safe,14.7,"To what extent would you say you are, or would...",2022
5,Fairly safe,48.5,"To what extent would you say you are, or would...",2022
6,Not particularly safe,24.3,"To what extent would you say you are, or would...",2022
7,Not safe at all,10.5,"To what extent would you say you are, or would...",2022
8,Strongly agree,8.5,To what extent do you agree or disagree that y...,2022
9,Tend to agree,25.6,To what extent do you agree or disagree that y...,2022


In [1109]:
df=dataframes['overall-crime-rate']
df = crimerate(df)
lydf=lydataframes['lastyearoverall-crime-rate']
lydf = crimerate(lydf)
overallcrime = pd.concat([df, lydf], ignore_index=True)
converttojson(overallcrime,'monthlyoverallcrimespercapita')
overallcrime

df=dataframes['hate-crime-rate']
df = crimerate(df, 'Hate crime')
lydf=lydataframes['lastyearhate-crime-rate']
lydf = crimerate(lydf, 'Hate crime')
hate = pd.concat([df, lydf], ignore_index=True)
converttojson(hate,'monthlyhatecrimespercapita')
hate

df=dataframes['domestic-abuse-crime-rate']
df = crimerate(df, 'Domestic Abuse')
lydf=lydataframes['lastyeardomestic-abuse-crime-rate']
lydf = crimerate(lydf, 'Domestic Abuse')
abuse = pd.concat([df, lydf], ignore_index=True)
converttojson(abuse,'monthlydomesticabusecrimespercapita')
abuse

df=dataframes['knife-crime-rate']
df = crimerate(df, 'Knife crime')
lydf=lydataframes['lastyearknife-crime-rate']
lydf = crimerate(lydf, 'Knife crime')
knives = pd.concat([df, lydf], ignore_index=True)
converttojson(knives,'monthlyknifecrimespercapita')
knives

df=dataframes['knife-crime-rate']
df = crimerate(df, 'Knife crime', "Knife Injury Victims (non DA 1-24)")
lydf=lydataframes['lastyearknife-crime-rate']
lydf = crimerate(lydf, 'Knife crime', "Knife Injury Victims (non DA 1-24)")
knives = pd.concat([df, lydf], ignore_index=True)
converttojson(knives,'monthlyknifecrimesvictimsU25percapita')
knives

C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:64: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


In [1112]:
violentcrime = crimerate(dataframes['violent-crime-rate'], 'Violence Against the Person')
lyviolentcrime = crimerate(lydataframes['lastyearviolent-crime-rate'], 'Violence Against the Person')
violentcrime = pd.concat([violentcrime, lyviolentcrime], ignore_index=True)
converttojson(violentcrime,'monthlyviolentcrimespercapita')
lyviolentcrime

burglar2023 = crimerate(dataframes['domestic-burglary-crime-rate'], 'Burglary', 'Domestic Burglary')
burglar2022 = crimerate(lydataframes['lastyeardomestic-burglary-crime-rate'], 'Burglary', 'Domestic Burglary')
burglar = pd.concat([burglar2023, burglar2022], ignore_index=True)
converttojson(burglar,'monthlydomesticburglarypercapita')
burglar

drugs2023 = crimerate(dataframes['drug-and-alcohol-misuse'], 'Drug Offences')
drugs2022 = crimerate(lydataframes['lastyeardrug-and-alcohol-misuse'], 'Drug Offences')
drugs = pd.concat([drugs2023, drugs2022], ignore_index=True)
converttojson(drugs,'monthlydrugoffensespercapita')
drugs

C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:64: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:64: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


DataFrame 'monthlyviolentcrimespercapita' saved as JSON file: Clean Data/monthlyviolentcrimespercapita.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:64: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:64: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


DataFrame 'monthlydomesticburglarypercapita' saved as JSON file: Clean Data/monthlydomesticburglarypercapita.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:64: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


DataFrame 'monthlydrugoffensespercapita' saved as JSON file: Clean Data/monthlydrugoffensespercapita.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1378232835.py:64: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  londondf = londondf.groupby(['Time period'], as_index=False).sum()


,AreaName,Population,Time period,Count,"Offences per 1,000 people"
0,Lambeth,317654,2023-04,104,0.3
1,Lambeth,317654,2023-05,116,0.4
2,Lambeth,317654,2023-06,104,0.3
3,Lambeth,317654,2023-07,113,0.4
4,Lambeth,317654,2023-08,125,0.4
...,...,...,...,...,...
564,London,8799728,2022-11,3531,0.4
565,London,8799728,2022-12,2899,0.3
566,London,8799728,2023-01,3738,0.4
567,London,8799728,2023-02,3047,0.3


In [821]:
# 2018 election
df = lydataframes['lastyeartotal-election-turnout-londonlambeth']
df = df.iloc[1:33,[1,2,6,7]]
df = firstrowtocolname(df)
column_mapping = {df.columns[0]: 'AreaName'}
df = df.rename(columns=column_mapping)
lydf = df[df['AreaName']=='Lambeth']

dflondon = lydataframes['lastyeartotal-election-turnout-lambethwards']
dflondon = dflondon.iloc[1:,[1,3,5,8,9]]
dflondon = firstrowtocolname(dflondon)

dflondon = dflondon[dflondon['Ward'].isin(['Total London Boroughs', 'England'])]

dflondon=dflondon.drop(dflondon.columns[[1]], axis=1, inplace=False)
dflondon

column_mapping = {dflondon.columns[0]: 'AreaName'}
dflondon = dflondon.rename(columns=column_mapping)
election_df2018 = pd.concat([lydf, dflondon], ignore_index=True)
election_df2018['Time period'] = 2018
column_mapping = {election_df2018.columns[1] :'Electorate',
                  election_df2018.columns[2] :'Ballot papers at the count',
                  election_df2018.columns[3]: 'Ballot box turnout'}
election_df2018 = election_df2018.rename(columns=column_mapping)

# 2022 election ---------------------------------------------------------------
df = dataframes['total-election-turnout-londonlambeth']
df = df.iloc[:2,:4]
column_mapping = {df.columns[0]: 'AreaName'}
df = df.rename(columns=column_mapping)

dfengland = dataframes['total-election-turnout-england']
column_mapping = {dfengland.columns[0]: 'AreaName'}
dfengland = dfengland.rename(columns=column_mapping)
dfengland = dfengland.iloc[0,:4]

dfwards = dataframes['total-election-turnout-lambethwards']
dfwards = dfwards.iloc[:,1:4]
dfwards['Ballot box turnout'] = dfwards.iloc[:,2].astype(int)/ dfwards.iloc[:,1].astype(int)
column_mapping = {dfwards.columns[0]: 'AreaName'}
dfwards = dfwards.rename(columns=column_mapping)

election_df = pd.concat([df, dfwards], ignore_index=True)
election_df = election_df.append(dfengland, ignore_index=True)
election_df['Time period'] = 2022
column_mapping = {election_df.columns[2] :'Ballot papers at the count',
                  election_df.columns[3]: 'Ballot box turnout'}
election_df = election_df.rename(columns=column_mapping)

# Combine both years ---------------------------------------------
election_df = pd.concat([election_df, election_df2018], ignore_index=True)
replace_dict = {'London Borough of Lambeth': 'Lambeth', 'Total London': 'London', 'Total':'England',
               'Total London Boroughs':'London'}
election_df['AreaName'] = election_df['AreaName'].replace(replace_dict)

election_df['Ballot box turnout'] = election_df['Ballot box turnout'].apply(lambda x: round(x * 100, 1))

converttojson(election_df, 'election2018-22')
election_df

DataFrame 'election2018-22' saved as JSON file: Clean Data/election2018-22.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\2666941033.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfwards['Ballot box turnout'] = dfwards.iloc[:,2].astype(int)/ dfwards.iloc[:,1].astype(int)
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\2666941033.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  election_df = election_df.append(dfengland, ignore_index=True)


,AreaName,Electorate,Ballot papers at the count,Ballot box turnout,Time period
0,Lambeth,226899,71925,31.7,2022
1,London,6036183,2151194,35.6,2022
2,Brixton Acre Lane,11042,3271,29.6,2022
3,Brixton North,11084,2869,25.9,2022
4,Brixton Rush Common,10646,3306,31.1,2022
5,Brixton Windrush,6845,1858,27.1,2022
6,Clapham Common & Abbeville,7357,2908,39.5,2022
7,Clapham East,7439,1744,23.4,2022
8,Clapham Park,9710,2497,25.7,2022
9,Clapham Town,11514,3562,30.9,2022


In [714]:
# Weekly earnings by sex ------------------------------------------------------------------
lydf= lydataframes['lastyearweekly-earnings-by-sex']
lydf.iloc[:,0] = lydf.iloc[:,0].str.split(':').str[-1]
lydf = lydf.iloc[1:,[0,1,3,5]]
lydf['Time period'] = 2022

df = dataframes['weekly-earnings-by-sex']
df.iloc[:,0] = df.iloc[:,0].str.split(':').str[-1]
df = df.iloc[1:,[0,1,3,5]]
df['Time period'] = 2023

filtered_df = pd.concat([lydf, df], ignore_index=True)
column_mapping = {filtered_df.columns[0]: 'AreaName'}
filtered_df = filtered_df.rename(columns=column_mapping)

melted_df = pd.melt(filtered_df, id_vars=['AreaName', 'Time period'], var_name='Sex', value_name='Value')
converttojson(melted_df, 'weekly-earnings-by-sex')
melted_df

DataFrame 'weekly-earnings-by-sex' saved as JSON file: Clean Data/weekly-earnings-by-sex.json


,AreaName,Time period,Sex,Value
0,Lambeth,2022,Male Full Time Workers,1019.3
1,England,2022,Male Full Time Workers,824.5
2,London,2022,Male Full Time Workers,1021.6
3,Lambeth,2023,Male Full Time Workers,1166.1
4,England,2023,Male Full Time Workers,869.7
5,London,2023,Male Full Time Workers,1065.6
6,Lambeth,2022,Female Full Time Workers,890.1
7,England,2022,Female Full Time Workers,686.4
8,London,2022,Female Full Time Workers,833.2
9,Lambeth,2023,Female Full Time Workers,924


In [681]:
# % ppl used Internet in the past 3 months
df=dataframes['internet-users']
df= df.drop(df.columns[[0,2]], axis=1, inplace=False)
df = df.iloc[:,:8]
column_mapping = {df.columns[0]: 'AreaName'}
df = df.rename(columns=column_mapping)
filtered_df = df[df.iloc[:,0].isin(['Lambeth','UK','London'])]
filtered_df= filtered_df[['AreaName',2019,2020]]
melted_df = pd.melt(filtered_df, id_vars='AreaName', var_name='Time period', value_name='Value')
converttojson(melted_df, 'internetusers2019-20')
melted_df

DataFrame 'internetusers2019-20' saved as JSON file: Clean Data/internetusers2019-20.json


,AreaName,Time period,Value
0,UK,2019,90.8
1,London,2019,93
2,Lambeth,2019,95.2
3,UK,2020,92.1
4,London,2020,94.9
5,Lambeth,2020,94.9


In [822]:
df = dataframes['sexual-orientation']
df_filtered = df[df['Sexual orientation'] != 'Straight or Heterosexual']
dataframes['sexual-orientation' ] = df_filtered
converttojson(dataframes['sexual-orientation'], 'sexual-orientation')

df = dataframes['gender-identity']
df_filtered = df[df['Gender identity'] != 'Gender identity the same as sex registered at birth']
dataframes['gender-identity'] = df_filtered
converttojson(dataframes['gender-identity'], 'gender-identity')

df= dataframes['working-age-population']
new_columns=[df.columns[0]]
for i, col in enumerate(df.columns):
        if pd.isna(col):
            new_col_name = f"{df.iloc[0, i]} {df.iloc[1, i]}"
            new_columns.append(new_col_name)
        else:
            continue
df.columns = new_columns
df = df[2:]
converttojson(df, 'working-age-population')
df

DataFrame 'sexual-orientation' saved as JSON file: Clean Data/sexual-orientation.json
DataFrame 'gender-identity' saved as JSON file: Clean Data/gender-identity.json
DataFrame 'working-age-population' saved as JSON file: Clean Data/working-age-population.json


,Population aged 16-64 (2021),Lambeth (Numbers),Lambeth (%),London (%),England (%)
2,All People Aged 16-64,241700,76.1,68.8,63
3,Males Aged 16-64,117200,76.1,68.8,63.3
4,Females Aged 16-64,124500,76.2,68.9,62.6


In [684]:
df = dataframes['personal-wellbeing']
df= df[['Geography','MeasureOfWellbeing', '2022-23', '2021-22']]
df = df.rename(columns={df.columns[0]: 'AreaName'})
melted_df = pd.melt(df, id_vars=['AreaName', 'MeasureOfWellbeing'], var_name='Time period', value_name='Average Value')
converttojson(melted_df, 'personal-wellbeing2021-23')
melted_df

DataFrame 'personal-wellbeing2021-23' saved as JSON file: Clean Data/personal-wellbeing2021-23.json


,AreaName,MeasureOfWellbeing,Time period,Average Value
0,England,Anxiety,2022-23,3.24
1,England,Happiness,2022-23,7.38
2,England,Life satisfaction,2022-23,7.44
3,England,Worthwhile,2022-23,7.73
4,Lambeth,Anxiety,2022-23,3.45
5,Lambeth,Happiness,2022-23,7.5
6,Lambeth,Life satisfaction,2022-23,7.63
7,Lambeth,Worthwhile,2022-23,7.75
8,London,Anxiety,2022-23,3.34
9,London,Happiness,2022-23,7.32


In [685]:
lambeth = dataframes['children-in-low-income-households-AHC-Lambeth'].iloc[:, [1] + list(range(11, 19))]
london = dataframes['children-in-low-income-households-AHC-London'].iloc[:, [0] + list(range(9, 17))]
df = pd.concat([lambeth, london], ignore_index=True)
df = df.rename(columns={df.columns[0]: 'AreaName'})
df.iloc[:,0] = df.iloc[:,0].str.strip()
filtered_df = df[df.iloc[:,0].isin(['Lambeth','England','London'])]
filtered_df= filtered_df[['AreaName','2020/21','2021/22']]
filtered_df.iloc[:, 1:3] = filtered_df.iloc[:, 1:3] * 100
melted_df = pd.melt(filtered_df, id_vars=['AreaName'], var_name='Time period', value_name='Percentage')

converttojson(melted_df, 'children-in-low-income-households-AHC')
melted_df

DataFrame 'children-in-low-income-households-AHC' saved as JSON file: Clean Data/children-in-low-income-households-AHC.json


,AreaName,Time period,Percentage
0,Lambeth,2020/21,38.787097
1,England,2020/21,30.625002
2,London,2020/21,37.239079
3,Lambeth,2021/22,35.512054
4,England,2021/22,30.777549
5,London,2021/22,32.852283


In [1115]:
# Cost of Living Resident survey ----------------------------------------------------------
q032 = dataframes['resident-survey-results---financial']
q4ay22_1 = dataframes['resident-survey-results---payforfood']
q4ay22_2 = dataframes['resident-survey-results---payforenergy']
q4ay22_3 = dataframes['resident-survey-results---payforrent']

# Join all DataFrames
surv_sat_f_cost = pd.concat([q032, q4ay22_1, q4ay22_2, q4ay22_3], ignore_index=True)

# Add a 'Year' column with value 2023
surv_sat_f_cost['Time period'] = 2023
surv_sat_f_cost.drop(surv_sat_f_cost.columns[2], axis=1, inplace=True)

# Remove anything between square brackets in Responses
surv_sat_f_cost['Response'] = surv_sat_f_cost['Response'].str.replace(r'\[.*\]', '')

# Make 'Response' an ordered categorical column with levels in the order of appearance
surv_sat_f_cost['Response'] = pd.Categorical(surv_sat_f_cost['Response'], categories=surv_sat_f_cost['Response'].unique(), ordered=True)
converttojson(surv_sat_f_cost,'survey_wellbeing_cost')
surv_sat_f_cost

# Last Year Cost of Living Resident survey ----------------------------------------------------------
q032 = lydataframes['lastyearresident-survey-results---financial']
q4ay22_1 = lydataframes['lastyearresident-survey-results---payforfood']
q4ay22_2 = lydataframes['lastyearresident-survey-results---payforenergy']
q4ay22_3 = lydataframes['lastyearresident-survey-results---payforrent']

# Join all DataFrames
lysurv_sat_f_cost = pd.concat([q032, q4ay22_1, q4ay22_2, q4ay22_3], ignore_index=True)

# Add a 'Year' column with value 2022
lysurv_sat_f_cost['Time period'] = 2022
lysurv_sat_f_cost

# Remove anything between square brackets in Responses
lysurv_sat_f_cost['Response'] = lysurv_sat_f_cost['Response'].str.replace(r'\[.*\]', '')

# Make 'Response' an ordered categorical column with levels in the order of appearance
lysurv_sat_f_cost['Response'] = pd.Categorical(lysurv_sat_f_cost['Response'], categories=lysurv_sat_f_cost['Response'].unique(), ordered=True)
converttojson(lysurv_sat_f_cost,'lastyearsurvey_wellbeing_cost')
lysurv_sat_f_cost

DataFrame 'survey_wellbeing_cost' saved as JSON file: Clean Data/survey_wellbeing_cost.json
DataFrame 'lastyearsurvey_wellbeing_cost' saved as JSON file: Clean Data/lastyearsurvey_wellbeing_cost.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\2998876437.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f_cost['Response'] = surv_sat_f_cost['Response'].str.replace(r'\[.*\]', '')
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\2998876437.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  lysurv_sat_f_cost['Response'] = lysurv_sat_f_cost['Response'].str.replace(r'\[.*\]', '')


,Response,Percentage of respondents,Question,Time period
0,Got worse,49.3,"Compared with this time last year, do you thin...",2022
1,Stayed the same,28.7,"Compared with this time last year, do you thin...",2022
2,Improved,19.9,"Compared with this time last year, do you thin...",2022
3,Very difficult,6.8,"Over the next year, how easy or difficult do y...",2022
4,Difficult,9.6,"Over the next year, how easy or difficult do y...",2022
5,Fairly difficult,10.8,"Over the next year, how easy or difficult do y...",2022
6,Fairly easy,31.2,"Over the next year, how easy or difficult do y...",2022
7,Easy,20.0,"Over the next year, how easy or difficult do y...",2022
8,Very easy,17.5,"Over the next year, how easy or difficult do y...",2022
9,Very difficult,18.0,"Over the next year, how easy or difficult do y...",2022


In [695]:
# Health Wellbeing Resident survey ----------------------------------------------------------
q029 = dataframes['wellbeing---resident-survey-worthwhile']
q030 = dataframes['wellbeing---resident-survey-anxiety']

# Add a space to each row of the Response column in q030
q030['Response'] = q030['Response'] + " "

# Join all DataFrames
surv_sat_f_health = pd.concat([q029, q030], ignore_index=True)
surv_sat_f_health.drop(surv_sat_f_health.columns[2], axis=1, inplace=True)

# Add a 'Year' column with value 2023
surv_sat_f_health['Time period'] = 2023

# Remove anything between square brackets in Responses
surv_sat_f_health['Response'] = surv_sat_f_health['Response'].str.replace(r'\[.*\]', '')

# Make 'Response' an ordered categorical column with levels in the order of appearance
surv_sat_f_health['Response'] = pd.Categorical(surv_sat_f_health['Response'], categories=surv_sat_f_health['Response'].unique(), ordered=True)
converttojson(surv_sat_f_health,'survey_wellbeing_health')
surv_sat_f_health

# Last year Health Wellbeing Resident survey ----------------------------------------------------------
q029 = lydataframes['lastyearwellbeing---resident-survey-worthwhile']
q030 = lydataframes['lastyearwellbeing---resident-survey-anxiety']

# Add a space to each row of the Response column in q030
q030['Response'] = q030['Response'] + " "

# Join all DataFrames
lastyearsurv_sat_f_health = pd.concat([q029, q030], ignore_index=True)

# Add a 'Year' column with value 2022
lastyearsurv_sat_f_health['Time period'] = 2022

# Remove anything between square brackets in Responses
lastyearsurv_sat_f_health['Response'] = lastyearsurv_sat_f_health['Response'].str.replace(r'\[.*\]', '')

# Make 'Response' an ordered categorical column with levels in the order of appearance
lastyearsurv_sat_f_health['Response'] = pd.Categorical(lastyearsurv_sat_f_health['Response'], categories=lastyearsurv_sat_f_health['Response'].unique(), ordered=True)

converttojson(lastyearsurv_sat_f_health,'lastyearsurvey_wellbeing_health')
lastyearsurv_sat_f_health

DataFrame 'survey_wellbeing_health' saved as JSON file: Clean Data/survey_wellbeing_health.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\1093869634.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  surv_sat_f_health['Response'] = surv_sat_f_health['Response'].str.replace(r'\[.*\]', '')


,Response,Percentage of respondents,Brixton Acre Lane,Brixton North,Brixton Rush Common,Brixton Windrush,Clapham Common & Abbeville,Clapham East,Clapham Park,Clapham Town,...,Streatham Common & Vale,Streatham Hill East,Streatham Hill West & Thornton,Streatham St Leonard's,Streatham Wells,Vauxhall,Waterloo & South Bank,West Dulwich,Question,Year
0,0 - Not at all worthwhile,1.0,0.0,0.0,4.0,0.0,2.7,0.0,0.0,1.3,...,1.6,0.0,1.6,1.0,3.0,0.0,0.0,0.0,"Overall, to what extent do you feel the things...",2023
1,1,0.1,0.0,0.9,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,"Overall, to what extent do you feel the things...",2023
2,2,1.1,0.8,0.0,3.3,0.0,0.0,0.0,3.6,0.0,...,0.9,3.0,3.9,0.0,0.0,1.0,0.0,0.0,"Overall, to what extent do you feel the things...",2023
3,3,1.2,1.7,3.2,0.0,0.0,0.0,1.3,0.0,0.0,...,3.5,0.0,2.1,2.6,0.0,0.0,0.8,3.3,"Overall, to what extent do you feel the things...",2023
4,4,1.3,3.4,1.3,5.5,0.0,0.0,2.3,4.1,0.0,...,0.9,0.0,2.8,0.0,0.5,0.0,0.0,0.0,"Overall, to what extent do you feel the things...",2023
5,5,6.2,4.8,4.2,9.2,9.3,3.3,2.4,9.7,4.3,...,6.0,3.9,8.5,7.4,11.2,5.7,9.7,11.6,"Overall, to what extent do you feel the things...",2023
6,6,6.9,10.7,12.6,11.9,4.6,7.4,13.4,5.4,5.7,...,4.4,9.9,1.0,8.4,8.1,11.1,4.6,0.5,"Overall, to what extent do you feel the things...",2023
7,7,16.0,14.4,22.5,6.6,20.3,23.9,10.5,14.2,22.5,...,9.3,16.6,19.3,15.4,13.0,9.8,13.0,16.2,"Overall, to what extent do you feel the things...",2023
8,8,23.3,25.0,20.1,25.8,34.3,26.7,20.4,20.0,16.6,...,22.5,18.8,31.5,24.6,17.5,17.6,13.8,30.4,"Overall, to what extent do you feel the things...",2023
9,9,11.1,12.2,2.5,7.6,6.5,6.1,13.3,11.4,11.7,...,18.9,9.5,10.8,11.3,14.8,22.7,7.5,11.6,"Overall, to what extent do you feel the things...",2023


In [1124]:
#Current year Council and Overall / Improving the Area survey results
q01 = dataframes['resident-survey-results---council-satisfaction-of-life']
q02 = dataframes['resident-survey-results---council-runs-things']
q03 = dataframes['resident-survey-results---council-value']
q04 = dataframes['resident-survey-results---council-informed']
q05 = dataframes['resident-survey-results---council-influence-decisions']
q06 = dataframes['resident-survey-results---improving-the-area'].iloc[:10]
q07 = dataframes['resident-survey-results---top3improve'].iloc[:10]

# Assuming q07 is a pandas DataFrame and Response is one of its columns
q07['Response'] = q07['Response'] + " "

q07['Response'] = q07['Response'].str.replace("Clean streets", "Clean streets ", regex=False)
q07['Response'] = q07['Response'].str.replace("The level of crime", "The level of crime ", regex=False)
surv_sat_f_general = pd.concat([q01, q02, q03, q04, q05], ignore_index=True)
surv_sat_f_general.drop(surv_sat_f_general.columns[2], axis=1, inplace=True)
surv_sat_f_general['Year']=2023

surv_sat_f_general['Response'] = pd.Categorical(surv_sat_f_general['Response'], categories=surv_sat_f_general['Response'].unique(), ordered=True)
converttojson(surv_sat_f_general, 'survey_data-overallandcouncil')

surv_sat_f_general = pd.concat([q06, q07], ignore_index=True)
surv_sat_f_general.drop(surv_sat_f_general.columns[2], axis=1, inplace=True)
surv_sat_f_general['Year']=2023

surv_sat_f_general['Response'] = pd.Categorical(surv_sat_f_general['Response'], categories=surv_sat_f_general['Response'].unique(), ordered=True)
converttojson(surv_sat_f_general, 'survey_data-improvingthearea')

#Last year Community - Council and Overall / Improving the Area survey results -------------------
q01 = lydataframes['lastyearresident-survey-results---council-satisfaction-of-life']
q02 = lydataframes['lastyearresident-survey-results---council-runs-things']
q03 = lydataframes['lastyearresident-survey-results---council-value']
q04 = lydataframes['lastyearresident-survey-results---council-informed']
q05 = lydataframes['lastyearresident-survey-results---council-influence-decisions']
q06 = lydataframes['lastyearresident-survey-results---improving-the-area'].iloc[:10]
q07 = lydataframes['lastyearresident-survey-results---top3improve'].iloc[:10]

# Assuming q07 is a pandas DataFrame and Response is one of its columns
q07['Response'] = q07['Response'] + " "

q07['Response'] = q07['Response'].str.replace("Clean streets", "Clean streets ", regex=False)
q07['Response'] = q07['Response'].str.replace("The level of crime", "The level of crime ", regex=False)
surv_sat_f_general = pd.concat([q01, q02, q03, q04, q05], ignore_index=True)
surv_sat_f_general['Year']=2022
surv_sat_f_general['Response'] = pd.Categorical(surv_sat_f_general['Response'], categories=surv_sat_f_general['Response'].unique(), ordered=True)
converttojson(surv_sat_f_general, 'lastyearsurvey_data-overallandcouncil')

surv_sat_f_general = pd.concat([q01, q02, q03, q04, q05, q06, q07], ignore_index=True)
surv_sat_f_general['Year']=2022
surv_sat_f_general['Response'] = pd.Categorical(surv_sat_f_general['Response'], categories=surv_sat_f_general['Response'].unique(), ordered=True)
converttojson(surv_sat_f_general, 'lastyearsurvey_data-improvingthearea')

C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\2361817921.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q07['Response'] = q07['Response'] + " "
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\2361817921.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q07['Response'] = q07['Response'].str.replace("Clean streets", "Clean streets ", regex=False)
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\2361817921.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

DataFrame 'survey_data-overallandcouncil' saved as JSON file: Clean Data/survey_data-overallandcouncil.json
DataFrame 'survey_data-improvingthearea' saved as JSON file: Clean Data/survey_data-improvingthearea.json
DataFrame 'lastyearsurvey_data-overallandcouncil' saved as JSON file: Clean Data/lastyearsurvey_data-overallandcouncil.json
DataFrame 'lastyearsurvey_data-improvingthearea' saved as JSON file: Clean Data/lastyearsurvey_data-improvingthearea.json


C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\2361817921.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q07['Response'] = q07['Response'] + " "
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\2361817921.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q07['Response'] = q07['Response'].str.replace("Clean streets", "Clean streets ", regex=False)
C:\Users\JKim\AppData\Local\Temp\ipykernel_19392\2361817921.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [697]:
# Current year Local area survey results
# Create DataFrames for each question
q08 = dataframes['resident-survey-results---local-area-diffbackgrounds'] 
q091 = dataframes['resident-survey-results---need-help']
q092 = dataframes['resident-survey-results---work-together']

# Concatenate (join) all dataframes
surv_sat_f_local = pd.concat([q08, q091, q092])
surv_sat_f_local.drop(surv_sat_f_local.columns[2], axis=1, inplace=True)


# Remove prefix from Question column
surv_sat_f_local['Question'] = surv_sat_f_local['Question'].str.replace("Please say how strongly you agree or disagree with each of the following statements - ", "")

# Create an ordered factor for the Response column
Response_unique_loc = surv_sat_f_local['Response'].unique()
surv_sat_f_local['Response'] = pd.Categorical(surv_sat_f_local['Response'], categories=Response_unique_loc, ordered=True)
converttojson(surv_sat_f_local, 'survey_data-localarea')

# Last year Local area survey results------------------------------------------------------
q08 = lydataframes['lastyearresident-survey-results---local-area-diffbackgrounds'] 
q091 = lydataframes['lastyearresident-survey-results---need-help']
q092 = lydataframes['lastyearresident-survey-results---work-together']

# Concatenate (join) all dataframes
surv_sat_f_local = pd.concat([q08, q091, q092])

# Remove prefix from Question column
surv_sat_f_local['Question'] = surv_sat_f_local['Question'].str.replace("Please say how strongly you agree or disagree with each of the following statements - ", "")

# Create an ordered factor for the Response column
Response_unique_loc = surv_sat_f_local['Response'].unique()
surv_sat_f_local['Response'] = pd.Categorical(surv_sat_f_local['Response'], categories=Response_unique_loc, ordered=True)
converttojson(surv_sat_f_local, 'lastyearsurvey_data-localarea')

DataFrame 'survey_data-localarea' saved as JSON file: Clean Data/survey_data-localarea.json
DataFrame 'lastyearsurvey_data-localarea' saved as JSON file: Clean Data/lastyearsurvey_data-localarea.json
